In [1]:
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from kerastuner import HyperModel, Hyperband


2.12.1
# GPUs Available:  1


In [2]:
# Experiment 4
# changes from experiment 2 (Attempt at MTL again in light that x1 x2 swap does improves learning)
# there is only 1 leakage ness factor - The assumption is that there is always atleast one leakage when the model is being used.
# Leakageness factor confirm if there exist 1 additional leakage as well.
# 1. interchanged x1 and x2 based on their Value
# removed column leak_1
# create a model 5 outputs

In [3]:
model_path = 'saved_model/Multi_leak/2_loss/Mid_structure_anchor/5 Output_to_transfer_withswap/'
project_name='multi_task_tuning_5coords_withswap'

In [4]:
with open("config_multi.yml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)


single_leakage, two_leakage = load_data(total_samples = cfg['experiment']['total_samples'])
# two_leakage["leak_1"] = 1
two_leakage["leak_2"] = 1

# single_leakage["leak_1"] = 1
single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
# use the idea of masking instead
data['x2'] = data['x2'].replace(np.nan, 8024)
data['y2'] = data['y2'].replace(np.nan, 2616.5)

In [5]:
data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'tot_residual_flow', 
       'total flow rate'
       ])

y = data[['x1', 'y1', 'x2', 'y2', 'leak_2']]
x = data.drop(['x1', 'y1', 'x2', 'y2', 'leak_2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 


# sort x1, y1 and x2, y2. coordinates with lowest x will take the position of x1
def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
y2_train = y_train[[ 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
y2_test = y_test[[ 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
y2_val = y_val[[ 'leak_2']]


# y1_test.to_csv('check1.csv')

y1_data = [y1_train, y1_val, y1_test]
y1_data_types = ['y1_train', 'y1_val', 'y1_test']
for y1_data_types, y1 in zip(y1_data_types, y1_data):
    y1_data_types = coords_swap(y1)

# print()

y1_columns = y1_train.columns
y2_columns = y2_train.columns
X_columns = X_train.columns

# y1_test.to_csv('check2.csv')


In [6]:

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y1_train = pd.DataFrame(y1_train, columns=y1_columns)
y1_test = pd.DataFrame(y1_test, columns=y1_columns)
y1_val = pd.DataFrame(y1_val, columns=y1_columns)

In [7]:
# y1_train['x2'] = y1_train['x2'].replace(np.nan, -5)
# y1_train['y2'] = y1_train['y2'].replace(np.nan, -5)

# y1_test['x2'] = y1_test['x2'].replace(np.nan, -5)
# y1_test['y2'] = y1_test['y2'].replace(np.nan, -5)

# y1_val['x2'] = y1_val['x2'].replace(np.nan, -5)
# y1_val['y2'] = y1_val['y2'].replace(np.nan, -5)
# Not sure if 0 is good enough or try generating a random number

# scaler_coords2 = StandardScaler()
# y2_train = scaler_coords2.fit_transform(y2_train)
# y2_test = scaler_coords2.fit_transform(y2_test)
# y2_val = scaler_coords2.transform(y2_val)

# y2_train = pd.DataFrame(y2_train, columns=y2_columns)
# y2_test = pd.DataFrame(y2_test, columns=y2_columns)
# y2_val = pd.DataFrame(y2_val, columns=y2_columns)

y2_train = y2_train.reset_index().drop(columns='sample_number')
y2_val = y2_val.reset_index().drop(columns='sample_number')
y2_test = y2_test.reset_index().drop(columns='sample_number')

# y_train = pd.concat([y1_train, y2_train], axis=1)
# y_test = pd.concat([y1_test, y2_test], axis=1)
# y_val = pd.concat([y1_val, y2_val], axis=1)

# scaler_flows = StandardScaler()
# X_train = scaler_flows.fit_transform(X_train)
# X_test = scaler_flows.transform(X_test)
# X_val = scaler_flows.transform(X_val)

In [8]:
X_train_np, y1_train_np, y2_train_np = X_train.values, y1_train.values, y2_train.values
X_val_np, y1_val_np, y2_val_np = X_val.values, y1_val.values, y2_val.values
X_test_np, y1_test_np, y2_test_np = X_test.values, y1_test.values, y2_test.values

# Create TensorFlow datasets from NumPy arrays.
batch_size = 32
buffer_size = len(X_train)  # Set the buffer size to the number of training examples for full shuffling.

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y1_train_np, y2_train_np ))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_np, y1_val_np, y2_val_np))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y1_test_np, y2_test_np))

# Shuffle, batch, and prefetch the training dataset.
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Batch the validation and test datasets.
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)


In [9]:
# y_train = {
#     "y1" : y1_train,
#     "y2" : y2_train
# }

# y_val = {
#     "y1" : y1_val,
#     "y2" : y2_val
# }

# y_test = {
#     "y1" : y1_test,
#     "y2" : y2_test
# }

losses = {
	"y1": "mse",
	"y2": 'binary_crossentropy'
    # "y2" : 'mse'
    }

metrics = {
    "y1": 'mae',
    "y2": 'mae'
    }

In [10]:
# Experiment 2 with Nadam

EPOCHS = 1000
# Define the multi-task HyperModel
class MultiTaskHyperModel(HyperModel):

    def build(self, hp):
        inputs = keras.Input(shape=(10,))
        shared_layer = inputs
        for i in range(hp.Int('num_layers', 1, 15)):
            shared_layer = layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                # activation=hp.Choice("activation", ["relu", 'elu']),
                activation = 'elu',
                # add elu
                kernel_initializer='he_uniform'
            )(shared_layer)


        task_layer1 = shared_layer
        for j in range(hp.Int(f'task_{i}_num_layers', 0, 10)):
            task_layer1 = layers.Dense(units=hp.Choice(f'task_{i}_layer_{j}_neurons', values=[4, 8, 16]), 
                                       activation='elu',
                                       kernel_initializer='he_uniform')(task_layer1)
        y1 = layers.Dense(4, name='y1', activation = 'linear', kernel_initializer='he_uniform')(task_layer1)

        task_layer2 = shared_layer
        for j in range(hp.Int(f'task_{i}_num_layers', 0, 10)):
            task_layer2 = layers.Dense(units=hp.Choice(f'task_{i}_layer_{j}_neurons', values=[4, 8, 16]),
                                        activation='elu',
                                        kernel_initializer='he_uniform')(task_layer2)
        y2 = layers.Dense(1, name='y2', activation = 'sigmoid', kernel_initializer='he_uniform')(task_layer2)

        outputs = [y1, y2]

        # loss1_weight = hp.Float("loss1_weight", min_value=1e-4, max_value=1, sampling="log")
        # loss1_weight = hp.Float("loss2_weight", min_value=1e-4, max_value=1, sampling="log")
        
        loss1_weight = hp.Choice('loss1_weight', values=[0.2, 0.4, 0.6, 0.8])
        # loss2_weight = hp.Choice('loss2_weight', values=[0.00005, 0.2, 0.4, 0.6, 0.8, 1.0])
        # to stop the model from reducing both these weightage to very low and hence reducing the total loss
        loss2_weight = 1 - loss1_weight

        lossWeights = {"y1": loss1_weight, "y2": loss2_weight}

        model = keras.Model(inputs=inputs, outputs=outputs)
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")

        # model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        #             loss="mse",  metrics='mae')
        
        model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=learning_rate),
                    loss=losses, loss_weights=lossWeights,
                    metrics = metrics)
        return model

# Create the multi-task HyperModel
multi_task_hypermodel = MultiTaskHyperModel()

# Define the Hyperband tuner
tuner = Hyperband(
    multi_task_hypermodel,
    objective =  kt.Objective("val_y1_mae", direction="min"),
    max_epochs=EPOCHS,
    factor=2,
    directory="../../tensorflow_log_files/studienarbeit/",
    project_name=project_name
)

# Perform hyperparameter search
tuner.search(train_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),
            #  X_train, y_train, 
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})), 
             verbose = 1,
             epochs=EPOCHS, shuffle = True)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:", best_hps)

# Build the best model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the best model on the full dataset
best_model.fit(train_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})),
            #  X_train, y_train, 
            #  validation_data = (X_val, y_val), 
            validation_data = val_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})), 
             verbose = 1, epochs=EPOCHS, shuffle = True)

Trial 2584 Complete [00h 04m 47s]
val_y1_mae: 0.5456151384657936

Best val_y1_mae So Far: 0.10773516657597541
Total elapsed time: 02h 06m 08s
INFO:tensorflow:Oracle triggered exit


Best Hyperparameters: <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7f202c131250>
Epoch 1/1000
20/20 [==============================] - 6s 24ms/step - loss: 0.5821 - y1_loss: 0.6698 - y2_loss: 0.5601 - y1_mae: 0.5929 - y2_mae: 0.3733 - val_loss: 0.3148 - val_y1_loss: 0.3653 - val_y2_loss: 0.3022 - val_y1_mae: 0.4738 - val_y2_mae: 0.2457
Epoch 2/1000
20/20 [==============================] - 0s 15ms/step - loss: 0.2866 - y1_loss: 0.2928 - y2_loss: 0.2850 - y1_mae: 0.3970 - y2_mae: 0.1939 - val_loss: 0.2084 - val_y1_loss: 0.4847 - val_y2_loss: 0.1393 - val_y1_mae: 0.5024 - val_y2_mae: 0.1199
Epoch 3/1000
20/20 [==============================] - 0s 14ms/step - loss: 0.0952 - y1_loss: 0.2358 - y2_loss: 0.0600 - y1_mae: 0.3454 - y2_mae: 0.0535 - val_loss: 0.0519 - val_y1_loss: 0.1988 - val_y2_loss: 0.0152 - val_y1_mae: 0.3268 - val_y2_mae: 0.0149
Epoch 4/1000
20/20 [==============================] - 0s 13ms/step - loss: 0.0461 - y1_loss: 0.1891 - y2_loss: 0.

In [11]:
best_model.save(model_path)

INFO:tensorflow:Assets written to: saved_model/Multi_leak/2_loss/Mid_structure_anchor/5 Output_to_transfer_withswap/assets


In [12]:
best_model.summary()
y_predictions = best_model.predict(test_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})))
results_train = best_model.evaluate(train_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})), verbose=1)
results_val = best_model.evaluate(val_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})), verbose=1)
results_test = best_model.evaluate(test_dataset.map(lambda x, y1, y2: (x, {'y1': y1, 'y2': y2})), verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 dense_15 (Dense)               (None, 512)          5632        ['input_2[0][0]']                
                                                                                                  
 dense_16 (Dense)               (None, 480)          246240      ['dense_15[0][0]']               
                                                                                                  
 dense_17 (Dense)               (None, 160)          76960       ['dense_16[0][0]']               
                                                                                            

In [13]:
y_pred = np.concatenate((y_predictions[0],y_predictions[1]), axis=1)
y_pred[:,-2][np.abs(y_pred[:,-2]) < 0.5] = 0
y_pred[:,-2][np.abs(y_pred[:,-2]) > 0.5] = 1
y_pred[:,-1][np.abs(y_pred[:,-1]) < 0.5] = 0
y_pred[:,-1][np.abs(y_pred[:,-1]) > 0.5] = 1
y_pred = pd.DataFrame(y_pred, columns=y_train.columns)
y1_pred_inverse = scaler_coords1.inverse_transform(y_pred[['x1', 'y1','x2', 'y2']])
y_pred[['x1', 'y1','x2', 'y2']] = pd.DataFrame(y1_pred_inverse,columns=['x1', 'y1','x2', 'y2'])
pd.concat([y_pred, y_test.reset_index().drop(columns='sample_number')], axis=1).to_csv(model_path+'predictions.csv')

In [16]:
from sklearn.metrics import mean_squared_error

mse_x1 = mean_squared_error(y_test['x1'], y_pred['x1'], squared=True)
mse_x2 = mean_squared_error(y_test['x2'], y_pred['x2'], squared=True)

mse_y1 = mean_squared_error(y_test['y1'], y_pred['y1'], squared=True)
mse_y2 = mean_squared_error(y_test['y2'], y_pred['y2'], squared=True)

# mse_leak1 = mean_squared_error(y_test['leak_1'], y_pred['leak_1'], squared=True)
mse_leak2 = mean_squared_error(y_test['leak_2'], y_pred['leak_2'], squared=True)

mse = [mse_x1, mse_x2, mse_y1, mse_y2, mse_leak2]

In [17]:
mse

[13673635.319496766,
 14659614.70321369,
 1249661.6812647914,
 1546425.9778987865,
 0.0]

In [ ]:


# idea to explore - 
# 1. Just use 1 loss function for the entire output
# 2. Give 2 loss function for each leakageness and coordinates (2a. Try the same without sigmoid function)
# 3.
# create a model first with 4 coordinate - 2 for each. 
# Then use that model as base model, add additional layers for leakageness and do transfer learning to do the 6 output case

# 4. 
# use masking feature of tensorflow (low priority) - # Use the idea of masking for the case where when we have to only 1 leakage 

# do i actually need objectness 1. since it is always present.i just need 1 objectness right ?
# coordinate are more important right - hence give more wightage to the coordinate loss function
# try auxiliary task and joint model
# remove bad quality data

In [ ]:
# swapping x1 and x2 in ascending order does improves accuracy
# found mistake in experiment 3 - that nan values in single leakage case were not actually updated.
# it was still blank. i suspect that it was assumed to be zero during the scaling process and scaled with the zero value.

In [ ]:
# new idea - put more weight on mse than cross entropy
# mse weights = 0.6 or 0.8
# entropy weights = 1 - mse weights